In [103]:
import pandas         as pd
import numpy          as np
import seaborn        as sns
import plotly.express as px
import ipywidgets     as widgets

from ipywidgets            import fixed
from matplotlib            import gridspec
from matplotlib            import pyplot as plt
from geopy.geocoders       import Nominatim
from IPython.core.display  import HTML
from IPython.display       import Image

In [104]:
# Funcoes 
def bt_intslider(Val,Min,Max,desc=''):
    
    bt = widgets.IntSlider(value = Val, 
                           min = Min,
                           max = Max, 
                           step = 1,
                           description=desc,
                           disable=False,
                           orientation='horizontal',
                           style = {'description_width': 'initial'})
    return bt    

# Criação de Botões
def bt_dropdown(data,x='',desc=''):
    
    bt1 = widgets.Dropdown(
    options=data[x].sort_values().unique().tolist(),
    description= desc,
    disabled=False,
    style={'description_width': 'initial'})
    
    return bt1   



# Estatística Descritiva
def num_metricas(num_attributes):
  #Central tendencian - mean, median

  ct1 = pd.DataFrame( num_attributes.apply(np.mean)).T
  ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

  #Dispersion - std, min, max, range, skew, kurtosis

  d1 = pd.DataFrame(num_attributes.apply(np.std)).T
  d2 = pd.DataFrame(num_attributes.apply(min)).T
  d3 = pd.DataFrame(num_attributes.apply(max)).T
  d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
  d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
  d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

  #Concatenate
  metrics = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
  metrics.columns = ['attributes','min', 'max', 'range', 'mean', 'median', 'std', 'skew','kurtosis']
  return metrics


def jupyter_settings():
    
    %matplotlib inline 
    #%pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr',False )
    pd.set_option('display.float_format',lambda x: '%.2f' % x)
    

    sns.set()

jupyter_settings()

In [32]:

data = pd.read_csv('../datasets/kc_house_data.csv')
# Garantir que o formato date é um datetime

data['date'] = pd.to_datetime( data['date'], format='%Y-%m-%d' )

# Suprimir notacao cientifica 

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)


In [33]:
# loading data into memory

data = pd.read_csv('../datasets/kc_house_data.csv')

# Garantir que o formato date é um datetime
data['date'] = pd.to_datetime( data['date'], format='%Y-%m-%d' )

# 2.3 1. Qual a média do preço de compra dos imóveis por “Nível”?

    - Nível 0 -> Preço entre R$ 0 e R$ 321.950
    - Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
    - Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
    - Nível 3 -> Acima de R$ 645.000

In [34]:

# Define level of prices 

data['level'] = data['price'].apply( lambda x : 'lvl_0' if x <= 321950 else
                                                'lvl_01'if (x >= 321950) & (x <= 450000) else
                                                'lvl_02'if (x >= 450000) & (x <= 645000) else
                                                'lvl_03'
                                                                                                )

In [35]:
data.sample(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level
19875,6056100370,2014-11-24,430000.00,3,2.25,2020,2750,2.00,0,0,...,1680,340,2008,0,98108,47.56,-122.30,1720,1546,lvl_01
7010,7751800080,2015-01-27,465000.00,3,1.50,1460,9879,1.00,0,0,...,1460,0,1956,0,98008,47.63,-122.13,1610,10050,lvl_02
16625,4139660430,2015-05-05,1200690.00,5,3.00,3640,28531,2.00,0,0,...,3640,0,1996,0,98006,47.55,-122.13,3330,17186,lvl_03
16811,414100280,2015-04-14,336000.00,2,1.00,1180,7200,1.00,0,0,...,1180,0,1949,0,98133,47.75,-122.34,1180,7200,lvl_01
7071,686400670,2015-04-14,678000.00,3,1.75,1670,7210,1.00,0,0,...,1670,0,1967,0,98008,47.63,-122.12,2200,7210,lvl_03
21297,7787920080,2014-06-16,492500.00,5,2.50,2570,9962,2.00,0,0,...,2570,0,2006,0,98019,47.73,-121.96,2890,9075,lvl_02
5049,7856410430,2014-05-30,1385000.00,6,2.75,5700,20000,1.00,0,4,...,2850,2850,1977,0,98006,47.56,-122.16,3690,15700,lvl_03
3354,7518508625,2015-04-16,900000.00,3,1.00,1560,3825,1.50,0,0,...,1390,170,1930,0,98117,47.68,-122.39,1700,5100,lvl_03
18322,7853310590,2014-05-29,658000.00,4,2.75,3310,6166,2.00,0,0,...,3310,0,2008,0,98065,47.52,-121.88,3200,7027,lvl_03
8647,5608000590,2014-07-14,929950.00,3,3.50,3790,10829,2.00,0,0,...,3790,0,1993,0,98027,47.55,-122.10,3620,10989,lvl_03


In [36]:
# Media de precos por cada Nivel. 

df = data[['level', 'price']].groupby( 'level' ).mean().reset_index()
df

,level,price
0,lvl_01,385677.01
1,lvl_02,539730.96
2,lvl_03,618517.93


In [37]:
data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'level'],
      dtype='object')

2.4 2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

In [38]:
data['size'] = data['sqft_living'].apply( lambda x :'size_0' if int(x <= 1427) else
                                                    'size_1' if int((x >= 1427)) & int((x <= 1910)) else
                                                    'size_2' if int((x >= 1910)) & int((x <= 2550)) else
                                                    'size_3')


In [39]:
# Media de precos por cada Nivel. 

df = data[['size', 'sqft_living']].groupby( 'size' ).mean().reset_index()
df

,size,sqft_living
0,size_0,1123.83
1,size_1,1664.96
2,size_2,2211.79
3,size_3,3329.61


# 2.5 3. Adicione as seguinte informações ao conjunto de dados original:

    - Place ID: Identificação da localização
    - OSM Type: Open Street Map type
    - Country: Nome do País
    - Country Code: Código do País

In [40]:
# 

In [41]:
# 

In [42]:
# 

In [43]:
# 

In [44]:
# 

In [45]:
# 

# 2.6 4. Adicione os seguinte filtros no Mapa:
    - Tamanho mínimo da área da sala de estar. OK
    
    - Número mínimo de banheiros. 
    - Valor Máximo do Preço. OK 
    - Tamanho máximo da área do porão.
    - Filtro das Condições do Imóvel.
    - Filtro por Ano de Construção.

In [46]:
data.columns
data.head(50)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size
0,7129300520,2014-10-13,221900.00,3,1.00,1180,5650,1.00,0,0,...,0,1955,0,98178,47.51,-122.26,1340,5650,lvl_03,size_0
1,6414100192,2014-12-09,538000.00,3,2.25,2570,7242,2.00,0,0,...,400,1951,1991,98125,47.72,-122.32,1690,7639,lvl_02,size_3
2,5631500400,2015-02-25,180000.00,2,1.00,770,10000,1.00,0,0,...,0,1933,0,98028,47.74,-122.23,2720,8062,lvl_03,size_0
3,2487200875,2014-12-09,604000.00,4,3.00,1960,5000,1.00,0,0,...,910,1965,0,98136,47.52,-122.39,1360,5000,lvl_02,size_2
4,1954400510,2015-02-18,510000.00,3,2.00,1680,8080,1.00,0,0,...,0,1987,0,98074,47.62,-122.05,1800,7503,lvl_02,size_1
5,7237550310,2014-05-12,1225000.00,4,4.50,5420,101930,1.00,0,0,...,1530,2001,0,98053,47.66,-122.00,4760,101930,lvl_03,size_3
6,1321400060,2014-06-27,257500.00,3,2.25,1715,6819,2.00,0,0,...,0,1995,0,98003,47.31,-122.33,2238,6819,lvl_03,size_1
7,2008000270,2015-01-15,291850.00,3,1.50,1060,9711,1.00,0,0,...,0,1963,0,98198,47.41,-122.31,1650,9711,lvl_03,size_0
8,2414600126,2015-04-15,229500.00,3,1.00,1780,7470,1.00,0,0,...,730,1960,0,98146,47.51,-122.34,1780,8113,lvl_03,size_1
9,3793500160,2015-03-12,323000.00,3,2.50,1890,6560,2.00,0,0,...,0,2003,0,98038,47.37,-122.03,2390,7570,lvl_01,size_1


In [47]:
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no')

In [48]:
data = pd.read_csv('../datasets/kc_house_data.csv')

In [49]:
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no')

# Price
limit = widgets.IntSlider(
    value = int(data['price'].mean()),
    min = int(data['price'].min()),
    max = int(data['price'].max()),
    step = 1,
    description='Maximun Price',
    disable=False,
    style={'description_width': 'initial'}
)

# waterfront_bar = widgets.Dropdown(
#     options = data['is_waterfront'].unique().tolist(),
#     value = 'yes',
#     description = 'Water View',
#     disable=False
# )

# Size 

livingroom_limit = widgets.IntSlider(
    value=int(data['sqft_living'].mean()),
    min = data['sqft_living'].min(),
    max = data['sqft_living'].max(),
    step=1,
    description=' Living Room Size',
    disable=False,
    style={'decription_width': 'initial'}
)

basement_limit = widgets.IntSlider(
    value=int(data['sqft_basement'].mean()),
    min = data['sqft_basement'].min(),
    max = data['sqft_basement'].max(),
    step=1,
    description=' Maximum Basement Size',
    disable=False,
    style={'decription_width': 'initial'}
)

bathroom_limit = widgets.IntSlider(
    value=int(data['bedrooms'].mean()),
    min = data['bedrooms'].min(),
    max = data['bedrooms'].max(),
    step=1,
    description=' Maximum Basement Size',
    disable=False,
    style={'decription_width': 'initial'}
)

condition_limit = widgets.IntSlider(
    value = int( data['condition'].mean() ),
    min = int(data['condition'].min()),
    max = int(data['condition'].max()),
    step = 1,
    description='Minimum condition',
    disable=False,
    style={'description_width': 'initial'}
)
year_limit = widgets.IntSlider(
    value = int( data['yr_built'].mean() ),
    min = int(data['yr_built'].min()),
    max = int(data['yr_built'].max()),
    step = 1,
    description='Year Built',
    disable=False,
)

def update_map (df, limit, livingroom_limit, bathroom_limit,basement_limit, condition_limit, year_limit):
    houses = df[(df['price'] < limit ) & # Maior por que buscamos o valor maximo 
            (df['sqft_living'] < livingroom_limit ) & 
            (df['bedrooms'] < bathroom_limit ) & 
            (df['is_waterfront'] < condition_limit ) & 
            (df['sqft_basement'] < basement_limit ) & 
            (df['yr_built'] < year_limit )] [['id','lat','long','price','sqft_living']].copy()
    # plot map 
    fig = px.scatter_mapbox(houses, 
                    lat="lat", 
                    lon="long",
                    size='price',
                    color_continuous_scale=px.colors.cyclical.IceFire, 
                    size_max=15, 
                    zoom=10,)

    fig.update_layout( mapbox_style ='open-street-map')
    fig.update_layout( height=600 , margin={ 'r':0, 'l': 0, 't':0, 'b': 0})

    fig.show()



# 2.6 4. Adicione os seguinte filtros no Mapa:
    - Tamanho mínimo da área da sala de estar. OK
    
    - Número mínimo de banheiros. 
    - Valor Máximo do Preço. OK 
    - Tamanho máximo da área do porão.
    - Filtro das Condições do Imóvel.
    - Filtro por Ano de Construção.

In [50]:
widgets.interactive( update_map,
    df=fixed( data ),
    limit=limit,
    livingroom_limit=livingroom_limit,
    bathroom_limit=bathroom_limit,
    basement_limit=basement_limit,
    condition_limit=condition_limit,
    year_limit=year_limit
)


interactive(children=(IntSlider(value=540088, description='Maximun Price', max=7700000, min=75000, style=Slide…

# 2.6 4. Adicione os seguinte filtros no Mapa:
    - Tamanho mínimo da área da sala de estar. OK
    - Número mínimo de banheiros. 
    - Valor Máximo do Preço. OK 
    - Tamanho máximo da área do porão.
    - Filtro das Condições do Imóvel.
    - Filtro por Ano de Construção.

#  2.7 5. Adicione os seguinte filtros no Dashboard:
    - Filtro por data disponível para compra.
    - Filtro por ano de renovação.
    - Filtro se possui vista para a água ou não.

In [110]:
df1 = df.copy()

In [113]:
data['year'] = pd.to_datetime(data['date']).dt.strftime( '%Y' )
data['date'] = pd.to_datetime(data['date']).dt.strftime( '%Y-%m-%d' )
data['year_week'] = pd.to_datetime(data['date']).dt.strftime( '%Y-%U')


In [107]:
date_limit = widgets.SelectionSlider(
    options= data['date'].sort_values().unique().tolist(),
    value= '2014-12-01',
    description='Max Avaliable Year',
    disabled=False,
    continous_update = False,
    style={'description_width': 'initial'},
    redout = True
)

renovated_limit = widgets.SelectionSlider(
    options= data['yr_renovated'].sort_values().unique().tolist(),
    value= 1950,
    description='Max Avaliable Year',
    disabled=False,
    continous_update = False,
    style={'description_width': 'initial'}
)

def update_dash( data, date, renovated):
    # Filtrar data
    df = data[(data['date'] <= date) & 
              (data['yr_renovated'] >= renovated)]


    fig = plt.figure( figsize=(21,12))
    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot( specs[0, :]) # Primeira linha
    ax2 = fig.add_subplot( specs[1, 0]) # Segunda Linha e Primeira coluna
    ax3 = fig.add_subplot( specs[1, 1]) # Segunda Linha e Segunda coluna
    
    by_year = data[['id', 'year']].groupby('year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'])
    ax1.set_title( 'Quantidade de imóvel disponível para compra', fontsize=16)
    
    by_day = data[['id', 'date']].groupby( 'date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'])
    ax2.set_title( 'Preço do imóvel por dia', fontsize=16)
    
    by_week_of_year = data[['id', 'year_week']].groupby( 'year_week').mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'])
    ax3.set_title( 'Preço do imóvel por semana do ano', fontsize=16)
    plt.xticks( rotation=50);


In [109]:
widgets.interactive( update_dash, data=fixed(data), date=date_limit, renovated=renovated_limit )

interactive(children=(SelectionSlider(description='Max Avaliable Year', index=227, options=('2014-05-02', '201…

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'is_waterfront', 'year',
       'year_week'],
      dtype='object')